In [19]:
# import pandas as pd

# # Đọc file CSV vào DataFrame
# df = pd.read_csv('data.csv')

# df = df.iloc[:, :1]

# # Ghi lại DataFrame vào file CSV
# df.to_csv('data.csv', index=False)


In [20]:
# import pandas as pd

# # Đọc dữ liệu từ file csv
# df = pd.read_csv('data_test_2.csv')

# # Tạo các cột trống
# columns = ['Team_Win']
# empty_df = pd.DataFrame(columns=columns)


# # Thêm các cột trống vào DataFrame ban đầu
# df = pd.concat([df, empty_df], axis=1)

# # Ghi DataFrame mới vào file csv
# df.to_csv('data_test_2.csv', index=False)


In [21]:
# import pandas as pd

# # Đọc file csv cũ
# df_old = pd.read_csv('data.csv')

# # Lấy 10 cột đầu tiên
# df_new = df_old.iloc[:10, :]

# # Lưu vào file csv mới
# df_new.to_csv('data_test_2.csv', index=False)


In [22]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json

In [23]:
# Đọc file csv
df = pd.read_csv('data_no_duplicates.csv')

# Lấy tất cả các link trong cột 'link'
links = df['link']

In [24]:
# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome()

In [25]:
lane_dict = {
    1: 'top_blue',
    2: 'top_red',
    3: 'jung_blue',
    4: 'jung_red',
    5: 'mid_blue',
    6: 'mid_red',
    7: 'ad_blue',
    8: 'ad_red',
    9: 'support_blue',
    10: 'support_red'
}


In [26]:
def get_champion_name(soup):
    i = 0
    rt = []
    div_tags = soup.find_all('div', {'class': 'relative'})
    for div_tag in div_tags:
        a_tag = div_tag.find('a')
        if a_tag:
            i= i+1
            img_tag = a_tag.find('img')
            if img_tag:
                champion_name = img_tag['alt']
                lane = lane_dict[i]
                rt.append((lane, champion_name))
    return rt

In [27]:
def get_Gold(soup):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        if tag.string and 'graphDD1' in tag.string:
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    # print(data[0])
                    return data[0]
    return None


In [28]:
def get_Turret(soup):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        if tag.string and 'graphDD41"' in tag.string :
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    blue = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    red = my_list2[15][1]
                    return blue, red
    return None

In [29]:
def get_KDA_Lane(soup, graph_ka, graph_d):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        if tag.string and graph_ka in tag.string :
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    ka_red = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    ka_blue = my_list2[15][1]
        if tag.string and graph_d in tag.string :
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    d_red = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    d_blue = my_list2[15][1]
    
    if (d_blue == 0): 
        d_blue = 1
    
    if (d_red == 0): 
        d_red = 1
    kda_blue = ka_blue/d_blue
    kda_red = ka_red/d_red
    return kda_blue, kda_red

In [30]:
def get_KDA_All_Lane(i, soup):
    kda_top_blue, kda_top_red = get_KDA_Lane(soup, 'graphDD11"', 'graphDD14"')
    kda_jungle_blue, kda_jungle_red = get_KDA_Lane(soup, 'graphDD69"', 'graphDD72"')
    kda_mid_blue, kda_mid_red = get_KDA_Lane(soup, 'graphDD127"', 'graphDD130"')
    kda_ad_blue, kda_ad_red = get_KDA_Lane(soup, 'graphDD185"', 'graphDD188"')
    kda_sp_blue, kda_sp_red = get_KDA_Lane(soup, 'graphDD243"', 'graphDD246"')
    df.loc[i, 'kda_top_blue'] = round(kda_top_blue, 2)
    df.loc[i, 'kda_top_red'] = round(kda_top_red, 2)
    df.loc[i, 'kda_jungle_blue'] = round(kda_jungle_blue, 2)
    df.loc[i, 'kda_jungle_red'] = round(kda_jungle_red, 2)
    df.loc[i, 'kda_mid_blue'] = round(kda_mid_blue, 2)
    df.loc[i, 'kda_mid_red'] = round(kda_mid_red, 2)
    df.loc[i, 'kda_ad_blue'] = round(kda_ad_blue, 2)
    df.loc[i, 'kda_ad_red'] = round(kda_ad_red, 2)
    df.loc[i, 'kda_sp_blue'] = round(kda_sp_blue, 2)
    df.loc[i, 'kda_sp_red'] = round(kda_sp_red, 2)

In [31]:
def get_Dragon_Count(soup):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        # in tag.string and 'graphDD380' not in tag.string and 'graphDD382' not in tag.string and 'graphDD419' not in tag.string
        if tag.string and 'graphDD38"' in tag.string:
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    blue = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    red = my_list2[15][1]
                    return blue, red
    return None

In [32]:
def get_Rift_Heralds_killed(soup):
    div_tag = soup.find('div', {'class': 'box matchBox'})
    th_blue_tags = div_tag.find_all('th', {'class': 'text-center no-padding-lateral'})
    
    blue_rift_heralds_tag = th_blue_tags[0].find('div', {'teamcounters-heralds-13'})
    blue_rift_heralds_value = blue_rift_heralds_tag.find_parent('div').text.strip()
    
    red_rift_heralds_tag = th_blue_tags[1].find('div', {'teamcounters-heralds-13'})
    red_rift_heralds_value = red_rift_heralds_tag.find_parent('div').text.strip()
    # print(blue_rift_heralds_value,"        ", red_rift_heralds_value)
    
    return blue_rift_heralds_value, red_rift_heralds_value

In [33]:
def get_teamWin(soup):
    teamWin = ""
    div_tag = soup.find('div', {'class': 'box matchBox'})
    th_blue_tag = div_tag.find('th', {'class': 'text-left no-padding-right'})
    
    blue_victory_tag = th_blue_tag.find('span', {'class': 'victory'})
    blue_defeat_tag = th_blue_tag.find('span', {'class': 'defeat'})
    # print(blue_victory_tag, "    ", blue_defeat_tag)
    # blue_victory_result = blue_victory_tag.text.strip()
    if (blue_victory_tag is not None) and (blue_defeat_tag is None):
        teamWin = "blue"
    else: 
        if  (blue_victory_tag is None) and (blue_defeat_tag is not None):
            teamWin = "red"
    # print(teamWin)
    return teamWin

In [34]:

i = 0
# Duyệt qua từng link và lấy thông tin
for link in links:
        # Mở trang web
    driver.get('https://www.leagueofgraphs.com/'+ link)

        # Chờ cho trang web tải xong
    driver.implicitly_wait(10)

        # Lấy HTML của trang web
    html = driver.page_source

        # Phân tích HTML bằng BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    lst = get_champion_name(soup)
    for my_tuple in lst:
        lane = my_tuple[0] # lấy phần tử đầu tiên trong tuple
        champion_name = my_tuple[1] # lấy phần tử thứ hai trong tuple
        df.loc[i,lane] = champion_name
    
    get_KDA_All_Lane(i, soup)
    
    my_string = ''.join(map(str, get_Gold(soup)))
    my_list = json.loads(my_string)
    gold_difference = my_list[15][1]
    df.loc[i, 'gold_difference'] = gold_difference

    turret_blue, turret_red = get_Turret(soup)
    df.loc[i, 'turret_blue'] = turret_blue
    df.loc[i, 'turret_red'] = turret_red

    dragon_blue, dragon_red= get_Dragon_Count(soup)
    df.loc[i, 'dragon_blue'] = dragon_blue
    df.loc[i, 'dragon_red'] = dragon_red

    blue_rift_heralds, red_rift_heralds = get_Rift_Heralds_killed(soup)
    df.loc[i, 'blue_heralds'] = blue_rift_heralds
    df.loc[i, 'red_heralds'] = red_rift_heralds

    df.loc[i,'team_win'] = get_teamWin(soup)
    print(str(i)+ " done");
    i = i + 1
    df.to_csv('data_no_duplicates.csv', index=False)

# Đóng trình duyệt
driver.quit()


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done


KeyboardInterrupt: 

In [ ]:
# import pandas as pd

# # Load the CSV file into a DataFrame
# df = pd.read_csv('data_no_duplicates.csv')

# # Add the new columns with empty values
# df['top_blue'] = ''
# df['top_red'] = ''
# df['jung_blue'] = ''
# df['jung_red'] = ''
# df['mid_blue'] = ''
# df['mid_red'] = ''
# df['ad_blue'] = ''
# df['ad_red'] = ''
# df['support_blue'] = ''
# df['support_red'] = ''
# df['gold_difference_at_20_minutes'] = ''
# df['turret_difference_at_20_minutes'] = ''
# df['dragon_difference_at_20_minutes'] = ''
# df['KDA_blue'] = ''
# df['KDA_red'] = ''
# df['Blue_Heralds_killed'] = ''
# df['Red_Heralds_killed'] = ''
# df['Team_Win'] = ''

# # Save the modified DataFrame back to the CSV file
# df.to_csv('data_no_duplicates.csv', index=False)
